In [1]:
with open('api.txt','r') as r:
    TEXT = r.read()

In [2]:
from ecommerce.src_base_scraper.scraper_builder import ScraperManagerBuilder
from ecommerce.src_base_scraper.requests_handler import RequestsHandler
from ecommerce.scraper_data.functions import get_pages,get_product,get_product_data

In [3]:
from ecommerce.data_parsing.iterator_parser import EcommerceFuncGenerator

In [4]:
handler = RequestsHandler.create_instance(None)
scrfunc = EcommerceFuncGenerator(requests_handler=handler,
                                 first_url = 'https://fragrancewholesale.co.uk/collections/chatler?page={2}',
                                 max_page = 6
                                 )
func = scrfunc.generate_url_function()
builder = ScraperManagerBuilder(requests_handler = handler,
                      initial_url='https://fragrancewholesale.co.uk/collections/chatler?page=1',
                      initial_node_function = get_pages,
                      node_functions=[get_product],
                      product_functions = get_product_data
                      )

In [5]:
scraper = builder.build()

In [6]:
a = scraper.get_data(parallel_processing_flag=False)

In [7]:
from ecommerce.src_base_scraper.product_info import ProductData

In [8]:
import sqlite3
from typing import Generator

# Your existing code for ProductInfo and ProductData here

def generate_table_sql(product_data: ProductData) -> str:
    """Generate the CREATE TABLE SQL statement based on the keys of product_info."""
    table_columns = ', '.join(f'{key} TEXT' for key in product_data.product_info_keys)
    return f'''
        CREATE TABLE IF NOT EXISTS product_table (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            url TEXT,
            raw_text_response TEXT,
            {table_columns}
        )
    '''

def save_to_database(generator: Generator[ProductData, None, None], database_path: str):
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()

    for product_data in generator:
        # Generate CREATE TABLE SQL statement based on the keys of product_info
        create_table_sql = generate_table_sql(product_data)
        cursor.execute(create_table_sql)

        # Insert data into the database
        cursor.execute('''
            INSERT INTO product_table (url, raw_text_response, {}) VALUES (?, ?, {})
        '''.format(', '.join(product_data.product_info_keys), ', '.join(['?'] * len(product_data.product_info_keys))),
        (product_data.url, product_data.raw_text_response, *product_data.product_info.values()))

    connection.commit()
    connection.close()

# Example usage:
# Assuming you have a generator named product_generator
# and you want to save the data to a SQLite database named 'products.db'


In [9]:
save_to_database(a,'products.db')

ProductData(url='https://fragrancewholesale.co.uk//collections/chatler/products/002-woman-eau-de-parfum-5-x-30ml-plus-1-free-tester', product_info={'name': '002 Woman Eau De Parfum 5 x 30ml Plus 1 free tester', 'price': '£8.95', 'description': <div class="description rte" itemprop="description">
<meta charset="utf-8"/>
<p data-mce-fragment="1"><span data-mce-fragment="1">Intoxicating rum distilled from sugarcane and aged in oak barrels, combines with exotic passion fruit in a fragrance filled with energy and glamour.</span></p>
<p data-mce-fragment="1"><span data-mce-fragment="1">Top Notes: Passion Fruit, Rum<br data-mce-fragment="1"/><br data-mce-fragment="1"/>Heart Notes: Musk, Gardenia<br data-mce-fragment="1"/><br data-mce-fragment="1"/>Base Notes: Tonka Bean, Vanilla</span></p>
</div>}, raw_text_response='')


TypeError: 'ProductData' object is not iterable